# The First Model - Aries


This model should at least perform the following:

* Read input data, such as model points, assumptions and projection paramters from input files, 
* Calculate death benefit cashflows and present values of the cashflows for all the model points,
* Aggregate the results across the model points, 
* Write aggregated results to an output file.

## Input Data and Files

This model should include thd followig types of input data:

* **Model point data**: A table whose records represent model points and columns represent plicy attributes
* **Mortality tables**: Multiple sets of mortality rates by sex and age
* **Lapse assumptions**: Multiple series of lapse rates by duration
* **Discount rates**: A series of annual interest rates by duration
* **Paramters**: Constant parameters and a table to associate assumptions with model points

Each type of the data above is stored in a separate file



### Model Point File

The model point file contains model point data in tabular format. 
The records of the file represent model points, and the columns represents model point attributes.
The first record contains a string column header whose values can be used as the attribute names.

#### File Format

* CSV

#### Columns

* **PolicyID**: Model point identifier (Integer or string)
* **ProductID**: Primary key used for assumption and paramter association and for result aggregation (Integer or string)
* **PlanID**: Secondary key used for assumption and paramter association and for result aggregation (Integer or string)
* **Sex**: `M` or `F` to indicate policy holder's sex (String)
* **IssueDate**: Date of issue (Integer, String or a Date type)
* **IssueAge**:  Policyholder's age at issue (Integer)
* **PolicyTerm**: Policy term in years (Integer)
* **PolicyCount**: Number of policies represented by the model point (Integer or Float)
* **SumAssured**: (Average) sum assured amount per policy in an unspecified currency (Integer or Float)

#### Number of Model Points

* 10000


In [87]:
import numpy as np
from numpy.random import default_rng  # Requires NumPy 1.17 or newer

rng = default_rng(12345)


# Number of Model Points
MPCount = 10000


# Product ID (String <= 10)

ProductID = [
    "TERM_1",
    "TERM_2"
]

product_id = np.fromiter(map(lambda i: ProductID[i], rng.integers(low=0, high=len(ProductID), size=MPCount)), np.dtype('<U10'))

# Plan ID (String <= 10)

PlanID = [
    "PLAN_A",
    "PLAN_B"
]

plan_id = np.fromiter(map(lambda i: PlanID[i], rng.integers(low=0, high=len(PlanID), size=MPCount)), np.dtype('<U10'))


# Sex (Char)

Sex = [
    "M",
    "F"
]

sex = np.fromiter(map(lambda i: Sex[i], rng.integers(low=0, high=len(Sex), size=MPCount)), np.dtype('<U1'))

# Issue Date (): 2010/1/1 - 2024/12/31 (Y/M/D)

date_begin = np.datetime64('2010-01-01')
date_end = np.datetime64('2025-01-01')
days = (date_end - date_begin).astype(int)

issue_date = date_begin + rng.integers(low=0, high=days, size=MPCount)

# Issue Age (Integer): 20 - 59 year old

issue_age = rng.integers(low=20, high=60, size=MPCount)

# Policy Term (Integer): 10, 15, 20 

policy_term = rng.integers(low=0, high=3, size=MPCount) * 5 + 10

# Policy Count (Integer): 1 - 100 

policy_count = rng.integers(low=0, high=101, size=MPCount)
 
# Sum Assured (Float): 10000 - 1000000  

sum_assured = (1000000 - 10000) * rng.random(size=MPCount) + 10000


In [84]:
import pandas as pd

attrs = [
    "ProductID",
    "PlanID",
    "Sex",
    "IssueDate",
    "IssueAge",
    "PolicyTerm",
    "PolicyCount",
    "SumAssured"
]

data = [
    product_id,
    plan_id,
    sex,
    issue_age,
    issue_date,
    policy_term,
    policy_count,
    sum_assured
]

ModelPoints = pd.DataFrame(dict(zip(attrs, data)), index=range(1, MPCount+1))
ModelPoints.index.name = "PolicyID"
ModelPoints

ProductID  PlanID Sex  IssueDate   IssueAge  PolicyTerm  PolicyCount  \
PolicyID                                                                        
1           TERM_2  PLAN_A   M         39 2012-10-13          20           69   
2           TERM_1  PLAN_A   F         53 2019-04-08          10           44   
3           TERM_2  PLAN_B   M         46 2016-04-17          15           87   
4           TERM_1  PLAN_B   F         30 2021-03-29          20           38   
5           TERM_1  PLAN_A   M         23 2016-10-31          15           20   
...            ...     ...  ..        ...        ...         ...          ...   
9996        TERM_2  PLAN_A   F         51 2023-09-03          20           66   
9997        TERM_1  PLAN_A   F         42 2020-06-13          20            9   
9998        TERM_2  PLAN_B   F         31 2013-08-23          10           17   
9999        TERM_1  PLAN_A   F         20 2012-07-05          20           97   
10000       TERM_1  PLAN_B   F         42 2017-09-29          10           94   

             SumAssured  
PolicyID                 
1         861683.506886  
2         568669.777455  
3         832762.135174  
4         718116.208315  
5         987991.680776  
...                 ...  
9996      260355.179719  
9997      813992.320821  
9998      108752.375284  
9999      101664.644951  
10000     187609.847405  

[10000 rows x 8 columns]

In [85]:
ModelPoints.to_csv("ModelPoints.csv")

### Mortality Table File

The mortality table file contains multiple mortality tables.
The tables are indentified by `MortID`.
Each table has mortality rates by age for male and female separately.


#### File Format

* Excel or CSV

### Lapse Assumption File

The lapse assumption file contains multiple set of lapse assumptions.
A lapse assumption is a series of lapse rates by policy duration.
Each assumption is identified by `LapseID`.


#### File Format

* Excel

### Discount Rates

#### File Format

* Excel or CSV

### Paramter File

This file should at least contain the following:

* Valuation Date
* Table to associate MortID and LapseID with combination of ProductID and PlanID

#### File Format

* Excel


## Output File


The output file should output the death benefit cashflows aggregted by `ProductID` and `PlanID`.